# This code generates a sample grid from the CoCIP grid model

Most of the code in this notebook is copied from [pycontrails.org](https://py.contrails.org/notebooks/CoCiPGrid.html).

In order to use the weather data API you need an account to access the Climate Data Store (CDS), where you will be given an API key. Click [here](https://cds.climate.copernicus.eu/how-to-api) for official instructions.
This KEY needs to be saved locally. 
You will also need to install the CDS API client by running `uv add cdsapi` in the terminal.

At the end of this notebook the results are saved as a dataset which will be used as the environment to run flights through.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pycontrails.core import MetDataset
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.cocipgrid import CocipGrid
from pycontrails.models.humidity_scaling import HistogramMatching
from pycontrails.models.ps_model import PSGrid

In [ ]:
# Set up time and spatial bounds for the model run
time_bounds = ("2024-01-01 00:00:00", "2024-01-01 23:00:00")
lon_bounds = (-8, 3)
lat_bounds = (49, 62)
# ideally would have  200 to 450 in 10 pha increments but not all levels available in ERA5
pressure_levels = (200, 225, 250, 300, 350, 400, 450)

In [ ]:
# Download meteorological data
era5 = ERA5(time_bounds, pressure_levels=pressure_levels, variables=CocipGrid.met_variables)
met = era5.open_metdataset()

era5_rad = ERA5(time_bounds, variables=CocipGrid.rad_variables)
rad = era5_rad.open_metdataset()

In [ ]:
# Model parameters
params = {
    "dt_integration": np.timedelta64(5, "m"),
    "max_age": np.timedelta64(10, "h"),
    # The humidity_scaling parameter is only used for ECMWF ERA5 data
    # See https://py.contrails.org/api/pycontrails.models.humidity_scaling.html#module-pycontrails.models.humidity_scaling
    "humidity_scaling": HistogramMatching(),
    # Use Poll-Schumann aircraft performance model adapted for grid calculations
    # See https://py.contrails.org/api/pycontrails.models.ps_model.PSGrid.html#pycontrails.models.ps_model.PSGrid
    "aircraft_performance": PSGrid(),
}

# Initialize CocipGrid model
cocip_grid = CocipGrid(met=met, rad=rad, params=params)

In [ ]:
# Create a grid source
coords = {
    "level": pressure_levels,
    "time": pd.date_range(time_bounds[0], time_bounds[1], freq="1h")[
        0:4
    ],  # run for first 4 hours of domain
    "longitude": np.arange(lon_bounds[0], lon_bounds[1], 1.0),
    "latitude": np.arange(lat_bounds[0], lat_bounds[1], 1.0),
}
grid_source = MetDataset.from_coords(**coords)

# Run CocipGrid model
result = cocip_grid.eval(source=grid_source)

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))
ef_per_m = result.data["ef_per_m"].isel(time=0, level=0)
ef_per_m.plot(x="longitude", y="latitude", vmin=-1e8, vmax=1e8, cmap="coolwarm")

plt.title("CocipGrid Energy Forcing")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

In [ ]:
# generate plots that show the ef_per_m at each time step and pressure level
for t in range(len(coords["time"])):
    for p in range(len(coords["level"])):
        plt.figure(figsize=(12, 8))
        ef_per_m = result.data["ef_per_m"].isel(time=t, level=p)
        ef_per_m.plot(x="longitude", y="latitude", vmin=-1e8, vmax=1e8, cmap="coolwarm")
        plt.title(
            f"CocipGrid Energy Forcing at time {coords['time'][t]} and level {coords['level'][p]} hPa"
        )
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()

In [ ]:
# show types in dataset
for var_name, var in result.data.variables.items():
    print(f"{var_name}: {var.dtype}")

# save dataset to netcdf
result.data.to_netcdf("cocipgrid_uk_adsb_jan_result.nc")